# Filtragem Colaborative 

## Baseada em Memória

In [ ]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#Storing the movie information into a pandas dataframe
movies_df = pd.read_csv('/content/drive/MyDrive/recomendacao/movies_small.csv')
#Storing the user information into a pandas dataframe
ratings_df = pd.read_csv('/content/drive/MyDrive/recomendacao/ratings_small.csv')

In [ ]:
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


Assim, cada filme tem um ID único, um título com seu ano de lançamento (que pode conter caracteres unicode) e vários gêneros diferentes no mesmo campo. Vamos remover o ano da coluna do título e colocá-lo em seu próprio, usando a prática função de extração que o Pandas possui.

Vamos remover o ano da coluna de título usando a função replace do pandas e armazená-lo em uma nova coluna de ano.

In [ ]:
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)

Este código está extraindo uma string de ano (em formato de quatro dígitos entre parênteses) do título de um filme em um DataFrame chamado "movies_df". A função "str.extract" está sendo usada para extrair o ano a partir da coluna "title" do DataFrame. O parâmetro "((\d\d\d\d))" é uma expressão regular que busca quatro dígitos entre parênteses. O parâmetro "expand=False" especifica que a saída deve ser uma série de strings, em vez de um dataframe.

In [ ]:
#Removendo os parenteses
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)

In [ ]:
#Removendo o ano da coluna título
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')

<ipython-input-7-409040d25172>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')


In [ ]:
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())

A função lambda é usada para remover espaços em branco no começo e no final de cada string na coluna "title" usando a função strip(). O resultado é uma nova coluna "title" com as strings limpas de espaços em branco.

In [ ]:
movies_df.head()

In [ ]:
ratings_df.head()

In [ ]:
ratings_df.shape

In [ ]:
ratings_df = ratings_df.drop('timestamp', 1)

<ipython-input-10-9d694ad7ac4e>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  ratings_df = ratings_df.drop('timestamp', 1)


Agora é hora de começar nosso trabalho nos sistemas de recomendação.

A primeira técnica que veremos é chamada Filtragem Colaborativa, também conhecida como Filtragem Usuário-Usuário. Conforme sugerido por seu nome alternativo, essa técnica usa outros usuários para recomendar itens ao usuário de entrada. Ele tenta encontrar usuários que tenham preferências e opiniões semelhantes à entrada e, em seguida, recomenda itens que eles gostaram para a entrada. Existem vários métodos para encontrar usuários semelhantes (mesmo alguns usando aprendizado de máquina), e o que usaremos aqui será baseado na função de correlação de Pearson.

O processo para criar um sistema de recomendação baseado no usuário é o seguinte:

- Selecione um usuário com os filmes que o usuário assistiu
- Com base em sua classificação dos filmes, encontre os principais $K$ vizinhos
- Obtenha o registro do filme assistido do usuário para cada vizinho
- Calcule uma pontuação de similaridade usando alguma fórmula
- Recomendar os itens com maior pontuação


In [ ]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


In [ ]:
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]

inputMovies = pd.merge(inputId, inputMovies)

inputMovies = inputMovies.drop(["genres","year"], 1)
inputMovies

<ipython-input-12-6b42ba7d4cb4>:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  inputMovies = inputMovies.drop(["genres","year"], 1)


,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,1968,"Breakfast Club, The",5.0


#Os usuários que viram os mesmos filmes
Agora, com os IDs do filme em nossa entrada, podemos obter o subconjunto de usuários que assistiram e revisaram os filmes em nossa entrada.

In [ ]:
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
userSubset.head()

,userId,movieId,rating
0,1,1,4.0
16,1,296,3.0
320,4,296,1.0
422,4,1968,4.0
516,5,1,4.0


In [ ]:
userSubsetGroup = userSubset.groupby(['userId'])

In [ ]:
type(userSubsetGroup)

In [ ]:
userSubsetGroup.get_group(4)

,userId,movieId,rating
320,4,296,1.0
422,4,1968,4.0


In [ ]:
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

Este código está ordenando uma lista chamada "userSubsetGroup" usando a função "sorted" com uma função de ordenação personalizada. A função de ordenação é uma função lambda que retorna o comprimento da segunda coluna (x[1]), e essa coluna é usada como chave de ordenação. O parâmetro "reverse=True" especifica que a lista deve ser ordenada em ordem decrescente, ou seja, os elementos com o maior comprimento de x[1] aparecerão primeiro na lista.

Em outras palavras, esse código está ordenando a lista "userSubsetGroup" pela quantidade de elementos na segunda coluna(x[1]), colocando os elementos com mais elementos na primeira posição.

In [ ]:
userSubsetGroup[0:3]

[(91,        userId  movieId  rating
  14121      91        1     4.0
  14122      91        2     3.0
  14173      91      296     4.5
  14316      91     1274     5.0
  14383      91     1968     3.0), (177,        userId  movieId  rating
  24900     177        1     5.0
  24901     177        2     3.5
  24930     177      296     5.0
  25069     177     1274     2.0
  25129     177     1968     3.5), (219,        userId  movieId  rating
  31524     219        1     3.5
  31525     219        2     2.5
  31554     219      296     4.0
  31628     219     1274     2.5
  31680     219     1968     3.0)]

## Semelhança de usuários com o usuário de entrada
Em seguida, vamos comparar todos os usuários (não todos !!!) com o usuário especificado e encontrar o que é mais semelhante.
Vamos descobrir a semelhança de cada usuário com o de entrada por meio do **Coeficiente de Correlação de Pearson**. É usado para medir a força de uma associação linear entre as duas variáveis. 

A fórmula para calcular a correlação de Pearson é:

$$r = \frac{\sum_{i=1}^n (x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum_{i=1}^n (x_i - \bar{x})^2}\sqrt{\sum_{i=1}^n (y_i - \bar{y})^2}}$$

onde:

- $x$ e $y$ são as variáveis de interesse
- $n$ é o número de observações
- $\bar{x}$ e $\bar{y}$ são as médias de $x$ e $y$, respectivamente
- $r$ é o coeficiente de correlação de Pearson



###Por que Correlação de Pearson?

A correlação de Pearson é invariante ao escalonamento, ou seja, multiplicar todos os elementos por uma constante diferente de zero ou adicionar qualquer constante a todos os elementos. Por exemplo, se você tiver dois vetores X e Y, então $pearson(X, Y) = pearson(X, 2 * Y + 3)$. 

Esta é uma propriedade muito importante em sistemas de recomendação porque, por exemplo, dois usuários podem classificar duas séries de itens de forma totalmente diferente em termos de taxas absolutas, mas seriam usuários semelhantes (ou seja, com ideias semelhantes) com taxas semelhantes em várias escalas.


Os valores dados pela fórmula variam de r = -1 a r = 1, onde 1 forma uma correlação direta entre as duas entidades (significa uma correlação positiva perfeita) e -1 forma uma correlação negativa perfeita.

No nosso caso, um 1 significa que os dois usuários têm gostos semelhantes, enquanto um -1 significa o oposto.

Vamos selecionar um subconjunto de usuários para iterar. Esse limite é imposto porque não queremos perder muito tempo passando por cada usuário.

Poderiamos iterar apenas em um subconjunto de usuários para poupar esforço computacional e quem sabe, selecionar os melhores usuários

In [ ]:
for name, group in userSubsetGroup:
  print(name,group)
  break

91        userId  movieId  rating
14121      91        1     4.0
14122      91        2     3.0
14173      91      296     4.5
14316      91     1274     5.0
14383      91     1968     3.0


In [ ]:
#Armazene a Correlação de Pearson em um dicionário, onde a chave é o ID do usuário e o valor é o coeficiente
pearsonCorrelationDict = {}

for name, group in userSubsetGroup:
    #Vamos começar classificando a entrada e o grupo de usuários atual para que os valores não se misturem mais tarde
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    #Obter o tamanho dos conjunto de usuários
    nRatings = len(group)
    #Obtenha as pontuações das críticas dos filmes que ambos têm em comum
    #Vai calcular a correlação entre os filmes que eles tem em comum!
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    
    # E então armazená-los em uma variável em formato de lista
    tempRatingList = temp_df['rating'].tolist()
    #Também vamos colocar as avaliações atuais do grupo de usuários em um formato de lista
    tempGroupList = group['rating'].tolist()
    #Agora vamos calcular a correlação de Pearson entre dois usuários, os chamados x e y

    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    #Se o denominador for diferente de zero, então divida, caso contrário, 0 correlação.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0

In [ ]:
pearsonCorrelationDict.items()

dict_items([(91, 0.43852900965351443), (177, 0.0), (219, 0.45124262819713973), (274, 0.716114874039432), (298, 0.9592712306918567), (414, 0.9376144618769914), (474, 0.11720180773462392), (477, 0.4385290096535153), (480, 0.7844645405527362), (483, 0.08006407690254357), (599, 0.7666866491579839), (608, 0.920736884379251), (50, 0.15713484026367722), (57, -0.7385489458759964), (68, 0.0), (103, 0.5222329678670935), (135, 0.8703882797784892), (182, 0.9428090415820635), (202, 0.5222329678670935), (217, 0.30151134457776363), (226, 0.9438798074485389), (288, 0.6005325641789633), (307, 0.9655810287305759), (318, 0.44486512077567225), (322, 0.5057805388588731), (330, 0.9035942578600878), (357, 0.5606119105813882), (434, 0.9864036607532465), (448, 0.30151134457776363), (469, 0.8164965809277261), (561, 0.5222329678670935), (600, 0.18442777839082938), (606, 0.9146591207600472), (610, -0.47140452079103173), (18, 1.0), (19, -0.5), (21, 0), (45, 0.5000000000000009), (63, -0.4999999999999982), (64, 0.0)

In [ ]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userId
0,0.438529,91
1,0.000000,177
2,0.451243,219
3,0.716115,274
4,0.959271,298


In [ ]:
pearsonDF[pearsonDF["userId"]==32]

,similarityIndex,userId
107,1.0,32


#Os $K$ principais usuários semelhantes ao usuário de entrada
Agora, vamos obter os 50 principais usuários que são mais semelhantes à entrada.

In [ ]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]

In [ ]:
topUsers.head()

,similarityIndex,userId
43,1.0,132
181,1.0,382
219,1.0,602
130,1.0,130
129,1.0,125


## Avaliação de usuários selecionados para todos os filmes
Faremos isso tomando a média ponderada das avaliações dos filmes usando a Correlação de Pearson como peso. Mas, para fazer isso, primeiro precisamos obter os filmes assistidos pelos usuários em nosso pearsonDF a partir do dataframe de classificações e, em seguida, armazenar sua correlação em uma nova coluna chamada `_similarityIndex`. Isso é obtido abaixo pela fusão dessas duas tabelas.

In [ ]:
topUsersRating=topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
topUsersRating.head()

,similarityIndex,userId,movieId,rating
0,1.0,132,1,2.0
1,1.0,132,17,3.0
2,1.0,132,29,2.0
3,1.0,132,32,3.0
4,1.0,132,34,1.5


Agora tudo o que precisamos fazer é simplesmente multiplicar a classificação do filme por seu peso (o índice de similaridade), somar as novas classificações e dividir pela soma dos pesos.

Podemos fazer isso facilmente simplesmente multiplicando duas colunas, agrupando o dataframe por movieId e dividindo duas colunas:

Ele mostra a ideia de todos os usuários semelhantes a filmes candidatos para o usuário de entrada:

In [ ]:
#Multiplies the similarity by the user's ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,movieId,rating,weightedRating
0,1.0,132,1,2.0,2.0
1,1.0,132,17,3.0,3.0
2,1.0,132,29,2.0,2.0
3,1.0,132,32,3.0,3.0
4,1.0,132,34,1.5,1.5


In [ ]:
#Applies a sum to the topUsers after grouping it up by userId
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,36.0,124.0
2,18.0,58.0
3,3.0,11.0
5,3.0,8.5
6,13.0,49.5


In [ ]:
#Creates an empty dataframe
recommendation_df = pd.DataFrame()
#Now we take the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
1,3.444444,1
2,3.222222,2
3,3.666667,3
5,2.833333,5
6,3.807692,6


In [ ]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,movieId
movieId,,
3035,5.0,3035
1956,5.0,1956
2522,5.0,2522
2495,5.0,2495
2477,5.0,2477
163386,5.0,163386
2455,5.0,2455
2450,5.0,2450
2427,5.0,2427


In [ ]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

,movieId,title,genres,year
1433,1956,Ordinary People,Drama,1980
1826,2427,"Thin Red Line, The",Action|Drama|War,1998
1842,2450,Howard the Duck,Adventure|Comedy|Sci-Fi,1986
1846,2455,"Fly, The",Drama|Horror|Sci-Fi|Thriller,1986
1865,2477,Firewalker,Adventure,1986
1877,2495,"Fantastic Planet, The (Planète sauvage, La)",Animation|Sci-Fi,1973
1898,2522,Airport '77,Drama,1977
2288,3035,Mister Roberts,Comedy|Drama|War,1955
9380,163112,Winnie the Pooh Goes Visiting,Animation,1971
9382,163386,Winnie the Pooh and the Day of Concern,Animation,1972


In [ ]:
inputMovies

In [ ]:
user_132 = ratings_df[ratings_df.userId==132]

In [ ]:
user_132[user_132.movieId.isin(inputMovies.movieId.unique())]

,userId,movieId,rating
19919,132,1,2.0
19946,132,296,4.0
20048,132,1968,4.0


# Usando Correlação de Pearson de Outra Forma

In [ ]:
movies_df = pd.read_csv('/content/drive/MyDrive/recomendacao/movies_small.csv')

In [ ]:
#Removendo o ano da coluna título
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')

<ipython-input-35-409040d25172>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')


In [ ]:
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())

In [ ]:
ratings_df = pd.read_csv('/content/drive/MyDrive/recomendacao/ratings_small.csv')

In [ ]:
df = pd.merge(movies_df, ratings_df, on = "movieId")

In [ ]:
df = df[["movieId", "title", "userId", "rating"]]

In [ ]:
df.head()

,movieId,title,userId,rating
0,1,Toy Story,1,4.0
1,1,Toy Story,5,4.0
2,1,Toy Story,7,4.5
3,1,Toy Story,15,2.5
4,1,Toy Story,17,4.5


In [ ]:
matrix_ratings = pd.pivot_table(data = df, index="title", columns = "userId", values="rating")

In [ ]:
matrix_ratings

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
'Hellboy': The Seeds of Creation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Round Midnight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Salem's Lot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
'Til There Was You,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.0,NaN,NaN,NaN,NaN,4.5,NaN,NaN
xXx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,2.0
xXx: State of the Union,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5


In [ ]:
userInput = [
            {'movieId': 1968,'title':'Breakfast Club, The','userId':611, 'rating':5},
            {'movieId': 1,'title':'Toy Story','userId':611, 'rating':3.5},
            {'movieId': 2,'title':'Jumanji','userId':611, 'rating':2},
            {'movieId': 296,'title':"Pulp Fiction",'userId':611, 'rating':5},
            {'movieId': 1274,'title':'Akira','userId':611, 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,movieId,title,userId,rating
0,1968,"Breakfast Club, The",611,5.0
1,1,Toy Story,611,3.5
2,2,Jumanji,611,2.0
3,296,Pulp Fiction,611,5.0
4,1274,Akira,611,4.5


In [ ]:
movies_df.set_index('title',inplace=True)

In [ ]:
movies_df.loc[['Breakfast Club, The','Toy Story', 'Jumanji', 'Pulp Fiction', 'Akira']]

,movieId,genres
title,,
"Breakfast Club, The",1968,Comedy|Drama
Toy Story,1,Adventure|Animation|Children|Comedy|Fantasy
Jumanji,2,Adventure|Children|Fantasy
Pulp Fiction,296,Comedy|Crime|Drama|Thriller
Akira,1274,Action|Adventure|Animation|Sci-Fi


In [ ]:
df_teste = pd.concat([df, inputMovies])

In [ ]:
matrix_ratings_teste = pd.pivot_table(data = df_teste, index="title", columns = "userId", values="rating")

In [ ]:
user_novo = matrix_ratings_teste[611]
similar_user_novo = matrix_ratings_teste.corrwith(user_novo)
corr_joao = pd.DataFrame(similar_user_novo, columns=["Corr_Pearson"])
corr_joao = corr_joao.sort_values(by="Corr_Pearson", ascending=False)
corr_joao = corr_joao.drop(labels=611, axis=0)
corr_joao.dropna(inplace=True)

/usr/local/lib/python3.8/dist-packages/numpy/lib/function_base.py:2683: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/usr/local/lib/python3.8/dist-packages/numpy/lib/function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


In [ ]:
corr_joao

,Corr_Pearson
userId,
169,1.0
124,1.0
125,1.0
130,1.0
132,1.0
...,...
509,-1.0
145,-1.0
137,-1.0


In [ ]:
corr_joao.loc[32]

Corr_Pearson    1.0
Name: 32, dtype: float64

In [ ]:
df_teste.head()

In [ ]:
count_ratings = pd.DataFrame(df_teste.groupby("userId")["rating"].count())

In [ ]:
completando = corr_joao.join(count_ratings)

In [ ]:
completando[completando["rating"] >=200].sort_values("Corr_Pearson", ascending = False).head(20)